## Approach 3: Bootstrapped Ensemble
The first ensemble approach tested whether simply training 5 independent models with randomly initialized weights would enable calculation of a variance for predictions on unlabeled samples maximized. This may have been a weak form of calculating variance. A better approach may be to make the models even more different from each other by bootstrapping - that is, using a random subset (e.g. 90%) of the data to train each model independently, then see what they disagree on. I was particularly encouraged to try this bootstrapped ensemble method when I noticed that the paper [Active learning-assisted directed evolution](https://www.nature.com/articles/s41467-025-55987-8) used this approach for their top performing DNN Ensemble method.

This will not improve my hypothesized main issue I discussed last time of ensuring diversity is high in the newly selected samples. I will work on that next time and apply that to this new bootstrapped ensemble along with the dropout method.

In [1]:
from scripts.data_utils import train_val_test_split
from scripts.config import (
    DATA_PATH, 
    SEQUENCE_COL, 
    SCORE_COL, 
    TOK_MODEL, 
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED,
)

training_pool, val_dataloader, test_dataloader = train_val_test_split(
    DATA_PATH,
    SEQUENCE_COL,
    SCORE_COL,
    TOK_MODEL,
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED
)

Compared to approach 1, the main thing that should change is that the train dataloader from acquire_new_batch should be subsetted from immediately before training a new model.

In [2]:
import torch
from torch.utils.data import Subset, DataLoader
import numpy as np

from scripts.training import initialize_and_train_new_model
from scripts.acquisition import get_pool_predictions

def get_bootstrap_sample(labeled_indices, pool_dataset, train_dataloader_batch_size):
    bootstrap_indices = np.random.choice(labeled_indices, size=int(0.9*len(labeled_indices)),replace=True)
    bootstrap_subset = Subset(pool_dataset, bootstrap_indices)
    bootstrap_dataloader = DataLoader(bootstrap_subset, batch_size=train_dataloader_batch_size, shuffle=True)
    return bootstrap_dataloader

def train_ensemble(
        n_models, 
        model_name, 
        approach,
        learning_rate,
        weight_decay,
        epochs,
        labeled_indices,
        train_dataloader_batch_size,
        pool_dataset, 
        pool_dataloader, 
        val_dataloader,
        patience
        ):
    
    # define list to store predictions as each model is trained then evaluated
    ensemble_predictions = []
    
    for i in range(n_models):
        print(f"\nTraining Model {i+1}...")
        # set a changing manual seed
        torch.manual_seed(i)
        torch.cuda.manual_seed(i)

        # get bootstrap sample from labeled dataset
        bootstrap_dataloader = get_bootstrap_sample(labeled_indices, pool_dataset, train_dataloader_batch_size)

        # initialize and train a new model
        model = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, bootstrap_dataloader, val_dataloader, patience)
        
        # get model predictions on pool dataloader, append to ensemble predictions list
        pool_preds = get_pool_predictions(model, pool_dataloader, )
        ensemble_predictions.append(pool_preds)

    # stack ensemble predictions to create tensor of shape (n_models, n_unlabeled_samples)
    ensemble_predictions = torch.stack(ensemble_predictions, dim=0)
    print("Ensemble training complete, submitting predictions for next cycle.")
    # return list of ensemble predictions
    return ensemble_predictions

In [3]:
from pathlib import Path
import pandas as pd

from scripts.acquisition import acquire_new_batch, get_variances
from scripts.training import initialize_and_train_new_model, test_model
from scripts.campaigns import run_standard_finetuning


def get_learning_curves(
        n_samples,
        initial_n_samples,
        n_samples_per_batch,
        model_name, 
        approach,
        learning_rate, 
        weight_decay, 
        epochs, 
        training_pool, 
        train_dataloader_batch_size,
        pool_dataloader_batch_size,
        val_dataloader, 
        test_dataloader,
        patience=5,
        n_models=5,
        results_path="active_vs_standard_learning_curves.csv"
):
    results_path = Path(results_path)
    results_dir = results_path.parent
    results_dir.mkdir(parents=True, exist_ok=True)

    # Load existing results if the file exists, otherwise start with a fresh DataFrame.
    if results_path.exists():
        all_results_df = pd.read_csv(results_path)
    else:
        all_results_df = pd.DataFrame()
    
    total_pool_size = len(training_pool)
    unlabeled_indices = np.arange(total_pool_size)
    labeled_indices = np.array([], dtype=np.int64)

    ensemble_predictions = None
    current_cycle = 1
    total_cycles = int(np.ceil((n_samples-initial_n_samples)/n_samples_per_batch)) + 1
    
    while len(labeled_indices) < n_samples and len(unlabeled_indices) > 0:
        print(f"\nCycle {current_cycle}/{total_cycles}\n-------------------------------------------------")

        # on the first cycle, choose random samples of initial_n_samples size
        if ensemble_predictions is None:
            print(f"Choosing initial {initial_n_samples} samples randomly...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=None
            )
        # each other time, use the n_samples_per_batch with acquisition scores to select
        else:
            scores = get_variances(ensemble_predictions, f"results/03_bootstrap_ensemble/variances{current_cycle}.csv")
            print(f"Selecting new data points...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=scores
            )
        
        # give message when loop ends
        if len(unlabeled_indices) == 0:
            print("Unlabeled pool is empty. Proceeding to final model training.")
            break
        
        # evaluate active vs standard
        final_results = []

        # active
        print(f"\nTraining and evaluating model using {len(labeled_indices)} actively selected samples...")
        model_active = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, train_dataloader, val_dataloader, patience, return_history=False)
        results_active = test_model(model_active, test_dataloader, return_results=True)
        results_active = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'active',
            **results_active
        }
        final_results.append(results_active)

        # standard
        print(f"\nTraining and evaluating model using {len(labeled_indices)} randomly selected samples...")
        model_standard, _ = run_standard_finetuning(len(labeled_indices), approach, model_name, train_dataloader_batch_size, learning_rate, weight_decay, epochs, training_pool, val_dataloader, patience)
        results_standard = test_model(model_standard, test_dataloader, return_results=True)
        results_standard = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'standard',
            **results_standard
        }
        final_results.append(results_standard)
        # save to disk each time to save progress
        results_df = pd.DataFrame(final_results)
        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)
        all_results_df.to_csv(results_path, index=False)
        print(f"Progress for experiment {current_cycle-1} appended to {results_path}")

        # if it's the last cycle, skip ensemble predictions
        if (current_cycle == total_cycles):
            print("Experiments complete.")
            break

        print("Starting ensemble training and pool evaluation...")
        ensemble_predictions = train_ensemble(n_models, model_name, approach, learning_rate, weight_decay, epochs, labeled_indices, train_dataloader_batch_size, training_pool, pool_dataloader, val_dataloader, patience)
    
        current_cycle += 1
    return all_results_df

In [4]:
from scripts.config import (
    MODEL_NAME,
    APPROACH,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EPOCHS,
    POOL_BATCH_SIZE,
    PATIENCE,
    N_MODELS,
)

get_learning_curves(
    n_samples=256,
    initial_n_samples=16,
    n_samples_per_batch=16,
    model_name=MODEL_NAME,
    approach=APPROACH,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    epochs=EPOCHS,
    training_pool=training_pool,
    train_dataloader_batch_size=BATCH_SIZE,
    pool_dataloader_batch_size=POOL_BATCH_SIZE,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    patience=PATIENCE,
    n_models=N_MODELS,
    results_path='results/03_bootstrap_ensemble/active_vs_standard_learning_curve.csv'
)


Cycle 1/16
-------------------------------------------------
Choosing initial 16 samples randomly...

Training and evaluating model using 16 actively selected samples...


/home/oscar/miniconda3/envs/plm-active-learning/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Training]:  68%|██████▊   | 34/50 [00:10<00:05,  3.12it/s]


Early stopping triggered after 10 epochs with no improvement.
Train Loss: 0.0453 | Val Loss: 0.1821 | SpearmanR: 0.4282


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 131.19it/s]



Training and evaluating model using 16 randomly selected samples...


...

[Training]:  82%|████████▏ | 41/50 [00:24<00:05,  1.66it/s]


Early stopping triggered after 10 epochs with no improvement.
Train Loss: 0.0167 | Val Loss: 0.1126 | SpearmanR: 0.7132


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 130.30it/s]


Progress for experiment 15 appended to results/03_bootstrap_ensemble/active_vs_standard_learning_curve.csv
Experiments complete.


,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.187841,0.356249,0.360150,0.188244
1,n_samples,0,16,standard,0.223101,0.200714,0.213891,0.224081
2,n_samples,0,16,active,0.194776,0.384558,0.423288,0.195900
3,n_samples,0,16,standard,0.216796,0.210195,0.281253,0.217824
4,n_samples,0,16,active,0.213148,0.225369,0.230617,0.213714
5,n_samples,0,16,standard,0.224002,0.310611,0.286941,0.223918
6,n_samples,0,16,active,0.211229,0.386833,0.378709,0.211804
7,n_samples,0,16,standard,0.203456,0.119613,0.136839,0.203401
8,n_samples,0,16,active,0.244507,0.272153,0.262033,0.244376
9,n_samples,0,16,standard,0.196090,-0.043903,-0.059994,0.196097


In [5]:
lc_df = pd.read_csv("results/03_bootstrap_ensemble/active_vs_standard_learning_curve.csv")
lc_df.head()

,changing_var,local_exp_idx,value,training_method,avg_test_loss,spearmanr,pearsonr,final_mse
0,n_samples,0,16,active,0.197894,0.382034,0.373106,0.198538
1,n_samples,0,16,standard,0.320916,0.270746,0.294286,0.320806
2,n_samples,1,32,active,0.184044,0.545784,0.502709,0.184418
3,n_samples,1,32,standard,0.173450,0.416550,0.422225,0.173986
4,n_samples,2,48,active,0.236712,0.378425,0.410424,0.237579


In [6]:
import plotly.express as px

fig = px.line(lc_df, 'value', 'spearmanr',color="training_method")

fig.update_layout(
    xaxis_title="Number of Training Samples",
    yaxis_title="Spearman Correlation Coefficient"
)

fig.show()

In [7]:
import pandas as pd
import glob
from pathlib import Path

file_pattern = "results/03_bootstrap_ensemble/variances*.csv"
variance_files = glob.glob(file_pattern)
variance_files.sort()

all_variances_dfs = []

for filepath in variance_files:
    temp_df = pd.read_csv(filepath)
    column_name = Path(filepath).stem
    temp_df = temp_df.rename(columns={'variance': column_name})
    all_variances_dfs.append(temp_df)

final_df = pd.concat(all_variances_dfs, axis=1)

final_df

,variances10,variances11,variances12,variances13,variances14,variances15,variances16,variances2,variances3,variances4,variances5,variances6,variances7,variances8,variances9
0,0.149963,0.011875,0.025784,0.021565,0.042797,0.023504,0.020772,0.013999,0.028303,0.051192,0.048048,0.027420,0.055021,0.014279,0.049370
1,0.050087,0.005050,0.020347,0.031385,0.019446,0.037039,0.030839,0.119323,0.064065,0.027289,0.024033,0.024117,0.022003,0.027090,0.043563
2,0.069689,0.014829,0.010998,0.004873,0.013083,0.016777,0.035694,0.028558,0.052594,0.060716,0.149823,0.034743,0.123635,0.090861,0.006818
3,0.030903,0.022928,0.014478,0.082185,0.103137,0.105877,0.029240,0.044814,0.054224,0.040302,0.059472,0.037716,0.021208,0.004541,0.026667
4,0.036251,0.021226,0.081336,0.022297,0.035207,0.022637,0.013253,0.083888,0.057288,0.017450,0.041651,0.015074,0.017717,0.039450,0.071926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032242,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.089120,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.029723,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.050000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
columns_to_plot = ['variances2', 'variances9', 'variances15']

# "Melt" the selected columns into a long-format DataFrame
melted_df = final_df[columns_to_plot].melt(
    var_name='Cycle',      # New column for the original column names
    value_name='Variance'  # New column for the variance values
)

fig = px.histogram(
    data_frame=melted_df,
    x='Variance',                                  
    color='Cycle',                                 
    barmode='overlay',                             
    opacity=0.65,                                  
    histnorm='probability density',                
    title='Distribution of Variances Across Active Learning Cycles'
)

fig.show()

Once again, the active learning campaign underperformed the standard fine-tuning procedure. Compared to the MC Dropout experiment, we can see that the variance distribution shrinks over time, suggesting that the uncertainty is better calibrated, but to truly test for calibration, one would need to measure the correlation between the variance and the error. Looking at this distrubtion, it makes me wonder if perhaps choosing the most variable sequences is just too noisy, and maybe randomly sampling from a top, given proportion of samples would help keep randomness while still biasing for higher variance. 

Some amount of this might be accomplished by ensuring diversity in the newly chosen batch, so this will be my next experiment.